In [1]:
from numba import cuda
from numba import njit, jit
import numpy as np

Тест cuda.jit

In [2]:
@cuda.jit
def increment_by_one(arr):
    pos = cuda.grid(1)
    if pos < arr.size:
        arr[pos] += 1

In [3]:
arr = np.zeros(10, dtype=int)
for i in range(len(arr)):
    arr[i] = i

In [4]:
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
threadsperblock = 32
blockspergrid = (arr.size + (threadsperblock - 1)) // threadsperblock

In [6]:
increment_by_one[blockspergrid, threadsperblock](arr)

In [7]:
arr

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [8]:
B_INC = 0.0000000001

In [9]:
NUM_NEIGHB = 27

def get_site(coord, L):
    """Get the site index from the 3-vector of coordinates."""
    # XXX: 3D hardcoded, can do N-D
    return coord[0] * L[1] * L[2] + coord[1] * L[2] + coord[2]


def get_coord(site, L):
    """Get the 3-vector of coordinates from the site index."""
    # XXX: 3D hardcoded, can do N-D
    x = site // (L[1]*L[2])
    yz = site % (L[1]*L[2])
    y = yz // L[2]
    z = yz % L[2]
    return [x, y, z]


def get_neighbors(site, L):
    neighb = set()
    x, y, z = get_coord(site, L)
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            for k in [-1, 0, 1]:
                x1 = (x + i) % L[0]
                y1 = (y + j) % L[1]
                z1 = (z + k) % L[2]
                neighb.add(get_site([x1, y1, z1], L))
    
    return list(neighb)
    
def tabulate_neighbors(L):
    """Tabulate the root-2 neighbors on the 3D cubic lattice with PBC."""
    Nsite = L[0]*L[1]*L[2]
    neighb = np.empty((Nsite, NUM_NEIGHB), dtype=int)
    for site in range(Nsite):
        neighb[site, :] = get_neighbors(site, L)
    return neighb



def tabulate_neighbors_small_L(L):
    """
    Tabulate the root-2 neighbors on the 3D cubic lattice with PBC.
    Must be used when L[0], L[1] or L[2] < 3 
    """
    Nsite = L[0]*L[1]*L[2]
    neighb = [[]] * Nsite
    for site in range(Nsite):
        neighb[site] = get_neighbors(site, L)
    return neighb

In [10]:
@cuda.jit
def culc_scores(grid, neighbors, scores, b):
    site = cuda.grid(1)
    if site < grid.size:
        for site1 in neighbors[site]:
            scores[site] += (1 - grid[site1])

        if grid[site] == 1:
            scores[site] *= b
            
@cuda.jit
def new_stratagies(grid, current, neighbors, scores, b):
    site = cuda.grid(1)
    if site < grid.size:
        best_site = site
        for site1 in neighbors[site]:
            if (scores[best_site] < scores[site1]):
                best_site = site1

        grid[site] = current[best_site]

@cuda.jit
def make_zeros(to_zeros):
    pos = cuda.grid(1)
    if pos < to_zeros.size:
        to_zeros[pos] = 0
        
        
@cuda.jit
def copy_ar(a, b):
    pos = cuda.grid(1)
    if pos < a.size:
        a[pos] = b[pos]

In [11]:
def evolve3D_3(grid, neighbors, b, num_steps=1):
    L = field.shape
    grid = grid.flatten().astype(int)
    SIZE = len(grid)
    _zeros = np.zeros((SIZE), dtype=float)
    _int_zeros = np.zeros((SIZE), dtype=int)
    evolve3D_impl(grid, neighbors, b, num_steps, _zeros, _int_zeros)
    
    return np.reshape(grid, L)

def evolve3D_impl(grid, neighbors, b, num_steps, _zeros, _int_zeros):
    
    ''' threads and blocks for cuda computation'''
    threadsperblock = 32
    blockspergrid = (arr.size + (threadsperblock - 1)) // threadsperblock
    
    
    ''' allocate global arrays '''
    current = cuda.to_device(_int_zeros)
    scores = cuda.to_device(_zeros)
    grid_global = cuda.to_device(grid)
    
    for step in range(num_steps):
        copy_ar[blockspergrid, threadsperblock](current, grid_global)
        make_zeros[blockspergrid, threadsperblock](scores)
        
        culc_scores[blockspergrid, threadsperblock](grid_global, neighbors, scores, b)
                
        new_stratagies[blockspergrid, threadsperblock](grid_global, current, neighbors, scores, b)
    grid = grid_global.copy_to_host()
    return grid

In [12]:
import timeit
rndm = np.random.RandomState(12345)
N = 1
Size = 60
L = (Size, Size, Size)
print(L)
neighbors = tabulate_neighbors(L)
field = (rndm.uniform(size=L) > 0.5).astype(int)
field = evolve3D_3(field, neighbors, 1.3, num_steps=N)

(60, 60, 60)


In [13]:
%timeit evolve3D_3(field, neighbors, 1.3, num_steps=100)

8.16 s ± 84.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
